In [46]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn as nn
from torch import optim as optim
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
ds=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Heart_Disease_Prediction.csv')
ds

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,52,1,3,172,199,1,0,162,0,0.5,1,0,7,Absence
266,44,1,2,120,263,0,0,173,0,0.0,1,0,7,Absence
267,56,0,2,140,294,0,2,153,0,1.3,2,0,3,Absence
268,57,1,4,140,192,0,0,148,0,0.4,2,0,6,Absence


In [4]:
ds.shape

(270, 14)

In [5]:
df=ds.iloc[:,[0,3,4,7,9,12,13]]
df.head()

,Age,BP,Cholesterol,Max HR,ST depression,Thallium,Heart Disease
0,70,130,322,109,2.4,3,Presence
1,67,115,564,160,1.6,7,Absence
2,57,124,261,141,0.3,7,Presence
3,64,128,263,105,0.2,7,Absence
4,74,120,269,121,0.2,3,Absence


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Age            270 non-null    int64  
 1   BP             270 non-null    int64  
 2   Cholesterol    270 non-null    int64  
 3   Max HR         270 non-null    int64  
 4   ST depression  270 non-null    float64
 5   Thallium       270 non-null    int64  
 6   Heart Disease  270 non-null    object 
dtypes: float64(1), int64(5), object(1)
memory usage: 14.9+ KB


In [6]:
x=df.iloc[:,0:5].values
y=df.iloc[:,6]

In [7]:
encoder=LabelEncoder()
y=encoder.fit_transform(y)

In [21]:
y

array([1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1])

In [22]:
x

array([[7.00e+01, 1.30e+02, 3.22e+02, 1.09e+02, 2.40e+00],
       [6.70e+01, 1.15e+02, 5.64e+02, 1.60e+02, 1.60e+00],
       [5.70e+01, 1.24e+02, 2.61e+02, 1.41e+02, 3.00e-01],
       ...,
       [5.60e+01, 1.40e+02, 2.94e+02, 1.53e+02, 1.30e+00],
       [5.70e+01, 1.40e+02, 1.92e+02, 1.48e+02, 4.00e-01],
       [6.70e+01, 1.60e+02, 2.86e+02, 1.08e+02, 1.50e+00]])

In [23]:
Y=y.reshape(-1,1)
Y

array([[1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
    

In [24]:
X_train,X_test,y_train,y_test=train_test_split(x,Y,test_size=0.2,random_state=42)

In [33]:
X_train,y_train

(array([[-0.63296991,  0.12509482,  0.40639114,  0.56612833, -0.9343038 ],
        [ 0.46519475,  2.14322204,  1.47854898, -0.39777057,  1.95963424],
        [-0.19370404, -0.09914154, -0.08095333,  1.04807779, -0.9343038 ],
        ...,
        [ 0.24556182, -0.65973243,  2.02437479,  0.60994192, -0.42360885],
        [-0.41333698, -1.78091422, -0.54880403, -0.26632981,  0.0870861 ],
        [-0.63296991, -0.09914154,  0.36740358,  0.60994192, -0.9343038 ]]),
 array([[0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
  

In [34]:
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


In [35]:
X_train_tensor,X_test_tensor,y_train_tensor,y_test_tensor=torch.from_numpy(X_train),torch.from_numpy(X_test),torch.from_numpy(y_train),torch.from_numpy(y_test)

In [36]:
X_train_tensor.shape

torch.Size([216, 5])

In [37]:
y_test_tensor

tensor([[1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1]])

In [38]:
y_train_tensor

tensor([[0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],

In [39]:
Hidden_layer=8

In [20]:
model=nn.Sequential(
    nn.Linear(X_train_tensor.shape[1],Hidden_layer),
    nn.ReLU(),
    nn.Linear(Hidden_layer,1),
    nn.Sigmoid()
)

In [41]:
criterion=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=0.01)

In [45]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32
epochs = 1000

train_dataset = TensorDataset(X_train_tensor.float(), y_train_tensor.float())
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * X_batch.size(0)

    epoch_loss /= len(train_loader.dataset)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")


Epoch 10, Loss: 0.3704
Epoch 20, Loss: 0.3703
Epoch 30, Loss: 0.3703
Epoch 40, Loss: 0.3703
Epoch 50, Loss: 0.3702
Epoch 60, Loss: 0.3703
Epoch 70, Loss: 0.3702
Epoch 80, Loss: 0.3703
Epoch 90, Loss: 0.3703
Epoch 100, Loss: 0.3703
Epoch 110, Loss: 0.3703
Epoch 120, Loss: 0.3703
Epoch 130, Loss: 0.3702
Epoch 140, Loss: 0.3703
Epoch 150, Loss: 0.3702
Epoch 160, Loss: 0.3703
Epoch 170, Loss: 0.3703
Epoch 180, Loss: 0.3703
Epoch 190, Loss: 0.3703
Epoch 200, Loss: 0.3703
Epoch 210, Loss: 0.3703
Epoch 220, Loss: 0.3703
Epoch 230, Loss: 0.3704
Epoch 240, Loss: 0.3703
Epoch 250, Loss: 0.3703
Epoch 260, Loss: 0.3703
Epoch 270, Loss: 0.3703
Epoch 280, Loss: 0.3703
Epoch 290, Loss: 0.3703
Epoch 300, Loss: 0.3703
Epoch 310, Loss: 0.3703
Epoch 320, Loss: 0.3702
Epoch 330, Loss: 0.3703
Epoch 340, Loss: 0.3702
Epoch 350, Loss: 0.3703
Epoch 360, Loss: 0.3703
Epoch 370, Loss: 0.3703
Epoch 380, Loss: 0.3702
Epoch 390, Loss: 0.3703
Epoch 400, Loss: 0.3703
Epoch 410, Loss: 0.3703
Epoch 420, Loss: 0.3703
E